In [2]:
import pandas as pd
import numpy as np
import scipy.stats
import re
import statsmodels.stats.multitest
import operator
import cptac.utils as u



/Users/Lindsey/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [41]:
sup18 = pd.read_csv("~/Documents/CPTAC_BC_SupplementaryTable18.csv")
sup18["phospho_site"] =  sup18["RefSeq"] + "_" + sup18["Phosphosite"]
sup18 = sup18[["Gene Name","FDR<5% PIK3CA mutation markers in all tumors and luminal tumors only (upregulated in mutated tumors)", "phospho_site","RefSeq"]]
sup18 = sup18.rename(columns = {"FDR<5% PIK3CA mutation markers in all tumors and luminal tumors only (upregulated in mutated tumors)" : "Significant", "Gene Name" : "Gene_Name"})

#sup18 = sup18.replace(r' ', "_", regex = True)
sup18

,Gene_Name,Significant,phospho_site,RefSeq
0,FAM184B,True,NP_056503_S976s,NP_056503
1,TRIM28,True,NP_005753_S489s,NP_005753
2,ARHGEF35,True,NP_001003702_S450s,NP_001003702
3,RPS6KA5,True,NP_004746_S212s,NP_004746
4,EIF2AK4,True,NP_001013725_S572s,NP_001013725
...,...,...,...,...
25856,FAM171A1,False,NP_001010924_S525s,NP_001010924
25857,TRIM2,False,NP_001123539_S424s S428s,NP_001123539
25858,MAP2,False,NP_002365_T1649t S1653s,NP_002365
25859,MAP2,False,NP_002365_S1155s,NP_002365


In [64]:

sup18_T = sup18.loc[sup18["Significant"] == True]
sup18_T = sup18_T.set_index("phospho_site")
sup18_T

,Gene_Name,Significant,RefSeq
phospho_site,,,
NP_056503_S976s,FAM184B,True,NP_056503
NP_005753_S489s,TRIM28,True,NP_005753
NP_001003702_S450s,ARHGEF35,True,NP_001003702
NP_004746_S212s,RPS6KA5,True,NP_004746
NP_001013725_S572s,EIF2AK4,True,NP_001013725
...,...,...,...
NP_001073922_S13s,INTS1,True,NP_001073922
NP_001191397_S596s,RBM10,True,NP_001191397
NP_001136202_S617s S620s,KCTD1,True,NP_001136202


# Get phosphoproteomic data from paper

In [62]:
df = pd.read_csv("~/Documents/CPTAC_BC_SupplementaryTable04.csv")
cols = [0,1,2,3,4,5,6,7,8,9,10,11,12,14,15,16,17,18,19,20,21,22,23,24,27]
df = df.drop(df.columns[cols],axis=1)
df["phospho_site"] =  df["accession_number"] + "_" + df["variableSites"]
df2 = df.set_index("phospho_site")
#df1 = df[df['phospho_site'].isin(cols_62)]
df2

,variableSites,accession_number,geneName,AO-A12D.01TCGA,C8-A131.01TCGA,AO-A12B.01TCGA,E2-A10A.02TCGA,BH-A18Q.02TCGA,C8-A130.02TCGA,C8-A138.03TCGA,...,AO-A12B.34TCGA,A2-A0SW.35TCGA,AO-A0JL.35TCGA,BH-A0BV.35TCGA,A2-A0YM.36TCGA,BH-A0C7.36TCGA,A2-A0SX.36TCGA,263d3f-I.CPTAC,blcdb9-I.CPTAC,c4155b-C.CPTAC
phospho_site,,,,,,,,,,,,,,,,,,,,,
NP_001611_S1007s,S1007s,NP_001611,AHNAK,-0.919089,-2.565861,0.177221,0.203602,-0.086607,-2.700983,1.301494,...,0.805357,4.389396,1.338505,3.450978,0.200324,1.728556,1.092898,-1.153228,0.234135,2.595100
NP_001611_S1010s,S1010s,NP_001611,AHNAK,-1.271514,-1.726458,-0.320742,-0.523359,-0.315189,-2.208631,1.820337,...,-0.124880,0.070029,0.495032,0.776968,1.017243,2.213385,0.444430,-0.331621,0.347756,2.527968
NP_001611_S1023s,S1023s,NP_001611,AHNAK,NaN,NaN,NaN,NaN,NaN,NaN,-0.302638,...,0.880172,0.812259,0.364182,1.626879,1.985442,-0.342381,1.767384,NaN,NaN,NaN
NP_001611_S1042s,S1042s,NP_001611,AHNAK,-2.298519,-0.370305,-1.169382,0.330244,0.819633,-0.419166,1.570435,...,-0.795385,0.797827,1.628386,1.382329,NaN,NaN,NaN,NaN,NaN,NaN
NP_001611_S1068s,S1068s,NP_001611,AHNAK,-0.802335,-2.023904,-1.223738,-0.465794,-1.585542,-1.411369,1.394314,...,-0.915370,2.065803,3.490470,2.797509,0.485597,2.001793,-0.352144,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NP_543009_T210t,T210t,NP_543009,GPR78,0.014945,2.483162,-0.324249,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NP_001017372_S217s T218t T224t,S217s T218t T224t,NP_001017372,SLC27A6,-0.698553,-1.693688,-0.175210,0.729908,0.613301,-0.054275,NaN,...,0.504689,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NP_954857_T47t,T47t,NP_954857,C19ORF48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
df62 = pd.merge(sup18_T, df2, on = "phospho_site", how = "inner")
df62.columns = df62.columns.str.replace(r'...TCGA', "", regex = True)
df62["Gene_Psite"] =  df62["Gene_Name"] + "_" + df62["variableSites"]
df62 = df62.set_index("Gene_Psite")
df62.head()

,Gene_Name,Significant,RefSeq,variableSites,accession_number,geneName,AO-A12D,C8-A131,AO-A12B,E2-A10A,...,AO-A12B,A2-A0SW,AO-A0JL,BH-A0BV,A2-A0YM,BH-A0C7,A2-A0SX,263d3f-I.CPTAC,blcdb9-I.CPTAC,c4155b-C.CPTAC
Gene_Psite,,,,,,,,,,,,,,,,,,,,,
FAM184B_S976s,FAM184B,True,NP_056503,S976s,NP_056503,FAM184B,-4.916841,-2.966657,-2.954333,-1.798008,...,-2.441299,NaN,NaN,NaN,-3.006837,-2.146746,-4.983482,3.717143,-1.413374,-2.761566
TRIM28_S489s,TRIM28,True,NP_005753,S489s,NP_005753,TRIM28,-2.624999,0.519513,-1.597210,1.724957,...,-1.151106,-0.828894,-2.297088,0.855144,-0.939327,0.467334,-0.956579,3.014077,2.552447,1.402929
ARHGEF35_S450s,ARHGEF35,True,NP_001003702,S450s,NP_001003702,ARHGEF35,-0.017487,0.365749,-0.562710,0.282548,...,-0.338030,-0.228925,-1.910580,-2.392154,NaN,NaN,NaN,NaN,NaN,NaN
RPS6KA5_S212s,RPS6KA5,True,NP_004746,S212s,NP_004746,RPS6KA5,0.157644,0.965682,0.026429,-0.107248,...,0.760187,-2.253563,-2.401767,-1.397919,0.168627,-0.127457,-1.482959,NaN,NaN,NaN
EIF2AK4_S572s,EIF2AK4,True,NP_001013725,S572s,NP_001013725,EIF2AK4,-0.162349,-0.836640,-1.160616,NaN,...,-0.801032,NaN,NaN,NaN,-0.631002,-0.632279,-1.815198,NaN,NaN,NaN


In [79]:
cols = ["Significant","RefSeq","accession_number","geneName","Gene_Name","variableSites"]
df62 = df62.drop(cols, axis = 1) 
df62.head()

,AO-A12D,C8-A131,AO-A12B,E2-A10A,BH-A18Q,C8-A130,C8-A138,E2-A154,BH-A0BZ,A8-A09I,...,AO-A12B,A2-A0SW,AO-A0JL,BH-A0BV,A2-A0YM,BH-A0C7,A2-A0SX,263d3f-I.CPTAC,blcdb9-I.CPTAC,c4155b-C.CPTAC
Gene_Psite,,,,,,,,,,,,,,,,,,,,,
FAM184B_S976s,-4.916841,-2.966657,-2.954333,-1.798008,-3.448592,2.449975,-4.629509,0.906083,-3.190797,-3.790538,...,-2.441299,NaN,NaN,NaN,-3.006837,-2.146746,-4.983482,3.717143,-1.413374,-2.761566
TRIM28_S489s,-2.624999,0.519513,-1.597210,1.724957,-0.394081,-2.106161,1.506175,2.703867,-0.561201,-1.867217,...,-1.151106,-0.828894,-2.297088,0.855144,-0.939327,0.467334,-0.956579,3.014077,2.552447,1.402929
ARHGEF35_S450s,-0.017487,0.365749,-0.562710,0.282548,-1.379211,-1.236422,-1.994830,0.811225,-2.304321,-2.450538,...,-0.338030,-0.228925,-1.910580,-2.392154,NaN,NaN,NaN,NaN,NaN,NaN
RPS6KA5_S212s,0.157644,0.965682,0.026429,-0.107248,-1.706914,-1.616308,-0.205057,2.482531,0.105787,-1.490049,...,0.760187,-2.253563,-2.401767,-1.397919,0.168627,-0.127457,-1.482959,NaN,NaN,NaN
EIF2AK4_S572s,-0.162349,-0.836640,-1.160616,NaN,NaN,NaN,-0.143177,0.811225,-0.591035,NaN,...,-0.801032,NaN,NaN,NaN,-0.631002,-0.632279,-1.815198,NaN,NaN,NaN


In [80]:
df2 = df62.T
df2.index.names = ['Sample ID']
#df2 = df2.dropna(how = "all", axis = 1)
df2.columns = df2.columns.str.replace(r' ', "_", regex = True)
df2.head()

Gene_Psite,FAM184B_S976s_,TRIM28_S489s_,ARHGEF35_S450s_,RPS6KA5_S212s_,EIF2AK4_S572s_,SRRT_S136s_,BCL9_S441s_,TP53BP1_S899s_,FAM21A_S836s_,MAPT_S726s_T731t_,...,FAM83H_S1024s_,WIZ_S289s_S294s_,DTNBP1_S11s_,C1ORF174_S28s_,RBM5_S18s_,INTS1_S13s_,RBM10_S596s_,KCTD1_S617s_S620s_,TAF3_S199s_,PRR12_S386s_
Sample ID,,,,,,,,,,,,,,,,,,,,,
AO-A12D,-4.916841,-2.624999,-0.017487,0.157644,-0.162349,-0.895306,1.418158,-0.270455,1.491670,-0.596934,...,-0.380722,2.190033,-0.011001,-0.843415,-0.367750,NaN,NaN,-0.780714,-2.134198,-1.223948
C8-A131,-2.966657,0.519513,0.365749,0.965682,-0.836640,1.011056,2.261338,0.980807,2.319315,-3.125463,...,2.619281,1.684090,1.195069,1.429497,1.227838,NaN,NaN,0.385914,-0.776142,0.176694
AO-A12B,-2.954333,-1.597210,-0.562710,0.026429,-1.160616,-0.859032,-1.181656,0.278918,-0.904621,-0.560956,...,0.015909,-1.371022,0.471791,-1.132561,-1.483239,NaN,NaN,0.263137,-2.068872,-1.453432
E2-A10A,-1.798008,1.724957,0.282548,-0.107248,NaN,-0.957562,-0.720724,2.945330,1.488119,-0.200996,...,0.025973,-4.562762,-0.074354,-3.620344,-3.924615,-2.360498,1.057205,-0.342441,0.022684,-1.283214
BH-A18Q,-3.448592,-0.394081,-1.379211,-1.706914,NaN,-1.496537,-1.413600,-0.731897,-1.033303,-4.629940,...,-0.723806,-2.083165,2.480397,-0.938229,-1.336731,-1.150629,-1.237611,-0.764263,-1.348868,-1.138491


# Get mutation data

In [81]:
mutations = pd.read_csv("~/Documents/CPTAC_BC_SupplementaryTable01.csv")
mutations = mutations.set_index("Sample ID")
mutations = mutations[["PIK3CA Mutation"]]
mutations = mutations.rename(columns = {"PIK3CA Mutation" : "PIK3CA_Mutation"})
mutations =  mutations.replace(np.nan, 'Wildtype', regex=True)
mutations

,PIK3CA_Mutation
Sample ID,
A2-A0CM,Wildtype
A2-A0D2,Wildtype
A2-A0EQ,Missense_Mutation|Somatic|p.H1047R
A2-A0EV,In_Frame_Ins|Somatic|p.E469in_frame_insDK
A2-A0EX,Missense_Mutation|Somatic|p.E545K
...,...
C8-A12W,Wildtype
D8-A13Y,Wildtype
E2-A10A,Wildtype


In [89]:
mutations.loc[mutations.PIK3CA_Mutation.str.contains('E542K')] = "Hotspot"
mutations.loc[mutations.PIK3CA_Mutation.str.contains('E545K')] = "Hotspot"
mutations.loc[mutations.PIK3CA_Mutation.str.contains('H1047R')] = "Hotspot"
mutations

,PIK3CA_Mutation
Sample ID,
A2-A0CM,Wildtype
A2-A0D2,Wildtype
A2-A0EQ,Hotspot
A2-A0EV,In_Frame_Ins|Somatic|p.E469in_frame_insDK
A2-A0EX,Hotspot
...,...
C8-A12W,Wildtype
D8-A13Y,Wildtype
E2-A10A,Wildtype


In [92]:
merged = pd.merge(df2, mutations, on = "Sample ID", how = "outer")
merged.head(30)

,FAM184B_S976s_,TRIM28_S489s_,ARHGEF35_S450s_,RPS6KA5_S212s_,EIF2AK4_S572s_,SRRT_S136s_,BCL9_S441s_,TP53BP1_S899s_,FAM21A_S836s_,MAPT_S726s_T731t_,...,WIZ_S289s_S294s_,DTNBP1_S11s_,C1ORF174_S28s_,RBM5_S18s_,INTS1_S13s_,RBM10_S596s_,KCTD1_S617s_S620s_,TAF3_S199s_,PRR12_S386s_,PIK3CA_Mutation
Sample ID,,,,,,,,,,,,,,,,,,,,,
AO-A12D,-4.916841,-2.624999,-0.017487,0.157644,-0.162349,-0.895306,1.418158,-0.270455,1.491670,-0.596934,...,2.190033,-0.011001,-0.843415,-0.367750,NaN,NaN,-0.780714,-2.134198,-1.223948,Wildtype
AO-A12D,-4.861437,-2.901710,-0.330563,-0.322597,-0.077631,-0.653201,1.087451,-0.894184,-0.384336,-0.999738,...,2.320247,0.358528,0.155386,-0.322597,NaN,1.055586,-0.714941,-2.268383,-0.834436,Wildtype
C8-A131,-2.966657,0.519513,0.365749,0.965682,-0.836640,1.011056,2.261338,0.980807,2.319315,-3.125463,...,1.684090,1.195069,1.429497,1.227838,NaN,NaN,0.385914,-0.776142,0.176694,Missense_Mutation|Somatic|p.R108H
C8-A131,NaN,0.710636,1.587940,0.295329,NaN,0.172457,0.956379,0.506669,NaN,-7.647102,...,1.007986,0.801561,NaN,NaN,0.450148,2.160523,NaN,-1.198792,0.270755,Missense_Mutation|Somatic|p.R108H
AO-A12B,-2.954333,-1.597210,-0.562710,0.026429,-1.160616,-0.859032,-1.181656,0.278918,-0.904621,-0.560956,...,-1.371022,0.471791,-1.132561,-1.483239,NaN,NaN,0.263137,-2.068872,-1.453432,Wildtype
AO-A12B,-2.441299,-1.151106,-0.338030,0.760187,-0.801032,-0.555415,-0.290036,NaN,-1.091819,-1.344494,...,-2.396128,0.853352,-1.711507,-0.597763,NaN,-1.352963,0.386115,-0.609055,-1.158164,Wildtype
E2-A10A,-1.798008,1.724957,0.282548,-0.107248,NaN,-0.957562,-0.720724,2.945330,1.488119,-0.200996,...,-4.562762,-0.074354,-3.620344,-3.924615,-2.360498,1.057205,-0.342441,0.022684,-1.283214,Wildtype
BH-A18Q,-3.448592,-0.394081,-1.379211,-1.706914,NaN,-1.496537,-1.413600,-0.731897,-1.033303,-4.629940,...,-2.083165,2.480397,-0.938229,-1.336731,-1.150629,-1.237611,-0.764263,-1.348868,-1.138491,Wildtype
C8-A130,2.449975,-2.106161,-1.236422,-1.616308,NaN,-1.573821,0.353103,-1.086467,-2.398574,1.360301,...,0.772977,-3.613210,-0.896523,-1.423866,-0.239220,-0.786556,1.840157,-0.446658,-0.786556,Hotspot


In [93]:
wt_hotspot = merged.loc[(merged['PIK3CA_Mutation'] == "Wildtype") |
                (merged['PIK3CA_Mutation'] == "Hotspot")]

In [97]:
test_cols = list(wt_hotspot.columns[:-1])
sigdf = u.wrap_ttest(wt_hotspot, "PIK3CA_Mutation", comparison_columns = test_cols , correction_method="fdr_bh", return_all=False, pval_return_corrected=False)

In [98]:
len(sigdf)

38

In [99]:
sigdf

,Comparison,P_Value
0,MAPT_S531s_T534t_,0.000011
1,MAPT_S411s_S437s_,0.000037
2,KCTD1_S617s_S620s_,0.000178
3,FAM184B_S976s_,0.000316
4,MAPT_T111t_,0.000444
5,MAPT_S531s_,0.000577
6,MAPT_S726s_T731t_,0.000965
7,MAPT_S733s_,0.001650
8,WWC3_S957s_,0.002570
9,ARID4B_S666s_,0.004713
